### LLMs com Langchain

In [1]:
import getpass
import os

In [ ]:
# descomente a linha específica caso precise resetar variável
# os.unsetenv("LANGCHAIN_API_KEY")
# os.unsetenv("OPENAI_API_KEY")
# os.unsetenv("GOOGLE_APPLICATION_CREDENTIALS")

In [7]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# informe api key p/ langchain smith tracking sevice
# os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

#### LLMs

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="Hi!")
]


In [4]:
# maneira mais indicada de ler o resultado
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

##### Open AI

In [12]:
from langchain_openai import ChatOpenAI

# garanta que o token está certo, pq acessar c/ token errado tb paga
# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [13]:
# pode passar modelo como parâmetro
model = ChatOpenAI(model="gpt-4") 

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f518f9ac790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f518f945610>, model_name='gpt-4', openai_api_key=SecretStr('**********'), openai_proxy='')

In [14]:
# invocando o modelo
result = model.invoke(messages)

In [24]:
# imprimir predict
print(result)

content='Ciao!' response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-447f1a66-fc25-4069-b959-56b9b8979669-0' usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23}
Ciao!


In [21]:
# maneira mais indicada de ler o resultado
print(parser.invoke(result))

Ciao!


In [25]:
# melhor ainda é criar um cadeia de chamadas
# invocando o model e o parser em sequência
chain = model | parser
chain.invoke(messages)

'Ciao!'

##### Vertex AI

In [6]:
# a biblioteca não está disponível no conda-forge
# podr isso tem que ser instalada com o cmd pip.
%pip install -qU langchain-google-vertexai

Note: you may need to restart the kernel to use updated packages.


In [14]:
from langchain_google_vertexai import ChatVertexAI

# informar aqui o caminho para as credenciais do google
# credencials deve ser de conta de serviço c/ permissão p/ VertexAI API
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/path/to/credentials"

model = ChatVertexAI(model="gemini-pro")

In [15]:
chain = model | parser
chain.invoke(messages)

'Ciao!'

#### Prompt Templates

In [16]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages([("system", system_template), ("user", "{text}")])

result = prompt_template.invoke({"language": "italian", "text": "Hi!"})

print(result)

print(result.to_messages())

messages=[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='Hi!')]
[SystemMessage(content='Translate the following into italian:'), HumanMessage(content='Hi!')]


In [17]:
# encadeando todos os componentes com pipe
chain = prompt_template | model | parser
chain.invoke({"language": "italian", "text": "Hi!"})

"Ciao! \n\nWhat would you like me to translate for you? \n\nJust provide the text or phrase in any language, and I'll do my best to translate it into Italian for you. \n"

#### Langserve

Agora execute o arquivo serve.py para servir o modelo com fastapi e langserve.

Isso irá subir um serviço de api na porta 8000: http://localhost:8000/chain/

##### Client

In [27]:
from langserve import RemoteRunnable

remote_chain = RemoteRunnable("http://localhost:8000/chain/")
remote_chain.invoke({"language": "italian", "text": "hi"})

'Ciao! Come posso aiutarti? \n\n(Hello! How can I help you?) \n'